In [41]:
!pip install torch torchquantum

  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchquantum-0.1.4-py3-none-any.whl (119 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached qiskit-0.32.1-py3-none-any.whl
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached qiskit_terra-0.18.3-cp37-cp3

In [2]:
!pip install pennylane pennylane-qiskit qiskit

     |████████████████████████████████| 1.0 MB 26.6 MB/s eta 0:00:01
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached retworkx-0.11.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
     |████████████████████████████████| 13.7 MB 50.7 MB/s eta 0:00:01
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
     |████████████████████████████████| 3.3 MB 84.1 MB/s eta 0:00:01
  Using cached qiskit_aer-0.11.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.2 MB)
  U

In [ ]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

## supernet

In [50]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'fmnist_4'
main_dir = './supernet/fmnist_4/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

inds = np.random.choice(len(x_test), 400, False)
x_test = x_test[inds]
y_test = y_test[inds]

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 2

device_name = 'ibmq_lima'
# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [12, 16, 20, 24]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0, 1], 'exp')
        
        val_exp_list = []
        
        for j in range(len(x_test)):
            val_exp_list.append(circ(x_test[j], curr_params))
            
        val_exps = np.array(val_exp_list)
        val_losses = np.sum(np.power(val_exps - y_test, 2), 1)
        val_loss = np.mean(val_losses)
        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)  
#     np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

ibmqfactory.load_account:WARNING:2022-10-02 14:00:41,708: Credentials are already in use. The existing account in the session will be replaced.


[101765  32307 129433] 0.2975 1.9897984790802001
[290807 211609  61758] 0.25 2.105005216598511
[284107 307257 260656] 0.2375 2.1334425449371337
[ 28323  29580 131678] 0.2825 2.080800714492798
[318774 311374   7658] 0.215 2.191361122131348
[144752  17549  69071] 0.2825 1.990298776626587
[147166  69454 126891] 0.2625 2.067657699584961
[  5875 136782 275953] 0.215 2.174487619400024
[149329 157548 331568] 0.2675 2.0413222217559817
[ 25467 242967 147419] 0.23 2.086214056015015
[212848  95199  97713] 0.22 2.0612947082519533
[136552  68778 318672] 0.2225 2.1677363872528077
[ 34573 295443  31216] 0.255 2.0048794460296633
[286148  71361 189745] 0.2975 2.0056074237823487
[270468 275639 312131] 0.27 2.0485662174224855
[121336 317720 130520] 0.275 2.163728380203247
[282998 285747  66468] 0.23 2.0393245124816897
[245839 301068 294313] 0.2025 2.228119926452637
[157512  83255 171482] 0.19 2.0555042266845702
[215744  59488 125146] 0.2325 2.106196060180664
[  3699 160246  49896] 0.23 2.1106607723236084

In [ ]:
from importlib import reload

import pennylane as qml

reload(qml)

In [ ]:
!python supernet/train_search_mnist.py --warmup_epochs 0 --steps 25000 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 3 --n_encode_layers 4 --save "12_params" --data "./experiment_data/mnist_2/" --save_dir "./supernet/mnist_2"

2022-09-23 05:02:09.093894: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-09-23 05:02:09.094011: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-09-23 05:02:09.123053: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(1600, 4, 4) (400, 4, 4)
subnet: [11524, 82582, 153173], expert_idx: 0
2022-09-23 05:02:21.814385: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 05:02:21.822319: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2499995000 Hz


In [ ]:
!python supernet/train_search_mnist.py --warmup_epochs 0 --steps 25000 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 4 --n_encode_layers 4 --save "16_params" --data "./experiment_data/mnist_2/" --save_dir "./supernet/mnist_2"

## train correlation circuits

In [ ]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'fmnist_2'
curr_dir = f'./experiment_data/{dataset}/trained_circuits/'

num_qubits = 4
num_embeds = 16

num_train_steps = 2500
num_test_data = 400

num_meas_qubits = 1

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=True)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(200, 400):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 1.8400399684906006
Step 101 | Loss: 0.9824213981628418
Step 201 | Loss: 0.9988893270492554
Step 301 | Loss: 0.9983148574829102
Step 401 | Loss: 1.000512719154358
Step 501 | Loss: 1.0025995969772339
Step 601 | Loss: 1.005479335784912
Step 701 | Loss: 1.0027689933776855
Step 801 | Loss: 0.9973701238632202
Step 901 | Loss: 0.979907751083374
Step 1001 | Loss: 1.0157033205032349
Step 1101 | Loss: 1.0111091136932373
Step 1201 | Loss: 1.0007538795471191
Step 1301 | Loss: 0.9992707371711731
Step 1401 | Loss: 0.996511697769165
Step 1501 | Loss: 1.0070708990097046
Step 1601 | Loss: 1.0002305507659912
Step 1701 | Loss: 0.9988362789154053
Step 1801 | Loss: 1.0056716203689575
Step 1901 | Loss: 0.9809823036193848
Step 2001 | Loss: 1.0019094944000244
Step 2101 | Loss: 0.9847947359085083
Step 2201 | Loss: 0.9974161386489868
Step 2301 | Loss: 0.9973360896110535
Step 2401 | Loss: 0.996767520904541
1.0092288692669085 0.5
Step 1 | Loss: 1.4163897037506104
Step 101 | Loss: 1.0265467166900635

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'fmnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

num_qubits = 4
num_embeds = 16
num_params = 18

device_name = 'ibmq_lima'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(400, 600):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
#             acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-10-24 04:06:50,178: Credentials are already in use. The existing account in the session will be replaced.


1.0197088297770074 1.3827621891047674 401
1.1272406146312668 1.4321240135449873 402
1.3025608854258954 1.36873736933033 403
1.218392629407745 1.2777175555019844 404


## humandesgn circs

In [75]:
from datasets import TorchDataset
from datasets_nt import load_dataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss
from create_noise_models import noisy_dev_from_backend
from create_human_design_circs import convert_human_design_circ_to_gate_circ

import pickle as pkl
import torch
import os
import numpy as np

embed_method = 'iqp'

dataset = 'vowel_2'
curr_dir = f'./human_design/{embed_method}_basic/{dataset}'

num_qubits = 4
num_embed_layers = 3
num_dataset_reps = 1

num_train_steps = 1000
num_test_data = 120

num_meas_qubits = 1

param_nums = [16, 20, 24, 28, 32]

if not os.path.exists(curr_dir):
    os.mkdir(curr_dir)

loss = TQMseLoss(num_meas_qubits, 4)
device = 'cpu'

train_data = TorchDataset(dataset, embed_method, num_dataset_reps, reshape_labels=True)
test_data = TorchDataset(dataset, embed_method, num_dataset_reps, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for param_num in param_nums:
    curr_param_dir = curr_dir + f'/{param_num}_params/'
    
    if not os.path.exists(curr_param_dir):
        os.mkdir(curr_param_dir)
    
    for i in range(1):    
        circ_dir = curr_param_dir

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_human_design_circ_to_gate_circ(num_qubits, embed_method, 'basic', num_embed_layers,
                                                                                                        param_num // num_qubits)
        
        np.savetxt(curr_param_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_param_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_param_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_param_dir + '/weights_bounds.txt', weights_bounds)
        
        losses_list = []
        accs_list = []

        for j in range(5):
            curr_train_dir = circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)

            model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
            opt = torch.optim.SGD(model.parameters(), lr=0.05)

            curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)

            print(curr_loss, curr_acc)

            losses_list.append(curr_loss)
            accs_list.append(curr_acc)
            
            torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        np.savetxt(circ_dir + '/val_losses.txt', losses_list)
        np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 1.059145450592041
Step 101 | Loss: 1.018198013305664
Step 201 | Loss: 1.1507503986358643
Step 301 | Loss: 0.9303577542304993
Step 401 | Loss: 0.9430369138717651
Step 501 | Loss: 0.7633569240570068
Step 601 | Loss: 0.9405395984649658
Step 701 | Loss: 0.9032794237136841
Step 801 | Loss: 0.9373906850814819
Step 901 | Loss: 0.9722119569778442
0.981750134136273 0.5666666666666667
Step 1 | Loss: 1.1858423948287964
Step 101 | Loss: 0.9825073480606079
Step 201 | Loss: 0.976843535900116
Step 301 | Loss: 1.0874971151351929
Step 401 | Loss: 0.8870750665664673
Step 501 | Loss: 1.0089274644851685
Step 601 | Loss: 0.8543026447296143
Step 701 | Loss: 0.9265348315238953
Step 801 | Loss: 1.0485655069351196
Step 901 | Loss: 0.8838073015213013
0.9668062135163681 0.5416666666666666
Step 1 | Loss: 0.8840807676315308
Step 101 | Loss: 0.8197826147079468
Step 201 | Loss: 0.7930959463119507
Step 301 | Loss: 0.8757045269012451
Step 401 | Loss: 0.8126855492591858
Step 501 | Loss: 0.851758956909179

In [162]:
from importlib import reload

import elivagar.inference.tq_circ_noisy_sim
import datasets_nt

reload(elivagar.inference.tq_circ_noisy_sim)
reload(datasets_nt)

<module 'datasets_nt' from '/root/datasets_nt.py'>

## noise metrics

In [ ]:
from importlib import reload
import elivagar.metric_computation.compute_clifford_nr

reload(elivagar.metric_computation.compute_clifford_nr)

## train our circuits

In [ ]:
from elivagar.training.train_circuits_from_predictors import train_elivagar_circuits

dataset = 'vowel_4'
device_name = 'ibmq_manila'
num_train_steps = 1000
num_params = 40
num_test_data = 120
num_qubits = 4
num_meas_qubits = 2
num_data_in_matrix = 64

circ_dir = f'./ours/{dataset}/{num_params}_params'

train_elivagar_circuits(circ_dir, dataset, device_name, num_train_steps, num_test_data, num_qubits, num_meas_qubits,
                            num_data_in_matrix, num_candidates_per_circuit=100, num_circuits=2500, num_runs=5)

Step 201 | Loss: 1.067293405532837
Step 301 | Loss: 1.1350668668746948
Step 401 | Loss: 1.2388997077941895
Step 501 | Loss: 1.2709476947784424
Step 601 | Loss: 1.065375566482544
Step 701 | Loss: 1.061983585357666
Step 801 | Loss: 0.9946576952934265
Step 901 | Loss: 1.2294495105743408
1.1726146715658836 0.6666666666666666
Step 1 | Loss: 1.9492011070251465
Step 101 | Loss: 1.334709644317627
Step 201 | Loss: 1.1559081077575684
Step 301 | Loss: 1.3054108619689941
Step 401 | Loss: 0.8566857576370239
Step 501 | Loss: 0.8089755177497864
Step 601 | Loss: 1.1627891063690186
Step 701 | Loss: 1.0729025602340698
Step 801 | Loss: 1.1424273252487183
Step 901 | Loss: 1.027600646018982
1.1446763477130883 0.6583333333333333
Step 1 | Loss: 2.4663705825805664
Step 101 | Loss: 1.4275672435760498
Step 201 | Loss: 1.1782761812210083
Step 301 | Loss: 1.1334788799285889
Step 401 | Loss: 1.1472740173339844
Step 501 | Loss: 0.9647291898727417
Step 601 | Loss: 1.0607974529266357
Step 701 | Loss: 1.21861767768859

In [ ]:
from elivagar.training.train_circuits_from_predictors import train_elivagar_circuits

dataset = 'fmnist_2'
device_name = None
embed_type = 'angle'
num_data_reps = 1
num_train_steps = 2500
num_params = 32
num_test_data = 400
num_qubits = 4
num_meas_qubits = 1
num_data_in_matrix = 32

circ_dir = f'./ours/{dataset}/{num_params}_params'

train_elivagar_circuits(circ_dir, dataset, embed_type, num_data_reps, device_name, num_train_steps, num_test_data, num_qubits, num_meas_qubits,
                            num_data_in_matrix, num_candidates_per_circuit=100, num_circuits=2500, num_runs=5)

[0.83158684 0.84251022 0.79097557 0.86144447 0.80526924 0.80498314
 0.82521248 0.82475471 0.74424171 0.86441612 0.78021812 0.80946541
 0.81977654 0.81616592 0.80131149 0.54882812 0.80265236 0.81085205
 0.8251133  0.80967522 0.80595016 0.83556938 0.85645294 0.78894806
 0.82696724]
0
[2313 1795  262 2370  881 1805  559  143  847 1244 1957 1563  604 1665
  853 1938 1366 2486 2019 1644 2293  255 1905  785  531   55 2002  817
 1296  323 1146 1672 2065  493 1164  111 1911 1068  529 2473  184 2319
  458 2389 1348  607  763 2258 1025  889 2133 1962 1010 1827 2188 1208
  153  409  968 1461 2471  996 1230 1693  231 1289 2226  904  730 1138
 1463 1602 1525  186  740 2064 1086  318 2171 1399 1904  669  404  447
  745  344 1151  449 1218 1658 1425 2330  897 1295 2310  849 1536 1780
 1765 2009] 1025
0.9101524353027344
Step 1 | Loss: 0.9380895495414734
Step 101 | Loss: 0.4010678827762604
Step 201 | Loss: 0.2963978946208954
Step 301 | Loss: 0.400852233171463
Step 401 | Loss: 0.5547229647636414
Step 50

In [146]:
from importlib import reload

import elivagar.training.train_circuits_from_predictors
import elivagar.metric_computation.compute_composite_scores
import elivagar.training.train_tq_circ_and_save
import create_gate_circs_np

reload(elivagar.training.train_circuits_from_predictors)
reload(elivagar.training.train_tq_circ_and_save)
reload(elivagar.metric_computation.compute_composite_scores)
reload(create_gate_circs_np)

<module 'create_gate_circs_np' from '/root/create_gate_circs_np.py'>

In [159]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ
import elivagar.inference.tq_circ_noisy_sim

reload(datasets)
reload(create_gate_circs)
reload(train_circ)
reload(elivagar.inference.tq_circ_noisy_sim)

<module 'elivagar.inference.tq_circ_noisy_sim' from '/root/elivagar/inference/tq_circ_noisy_sim.py'>

In [ ]:
from elivagar.inference.tq_circ_noisy_sim import run_noisy_inference_for_tq_circuits

dataset = 'mnist_2'
embed_type = 'angle'
num_data_reps = 1
num_test_samples = 400

num_qubits = 4
num_embeds = 16
meas_qubits = [0]
num_circs = 1000
num_runs = 5

device_name = 'ibmq_manila'

circ_dir = f'./experiment_data/{dataset}/trained_circuits'
circ_prefix = 'circ'

run_noisy_inference_for_tq_circuits(circ_dir, circ_prefix, num_circs, num_runs, num_qubits, meas_qubits, device_name, dataset,
                                        embed_type, num_data_reps, num_test_samples=None, human_design=False, compute_noiseless=False)

ibmqfactory.load_account:WARNING:2022-11-20 18:45:36,949: Credentials are already in use. The existing account in the session will be replaced.


0.43807296752929686 0.865


## perf metric

In [124]:
from importlib import reload

import metrics
import create_gate_circs

reload(metrics)
reload(create_gate_circs)

<module 'create_gate_circs' from '/root/create_gate_circs.py'>

In [125]:
from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets import load_dataset

# ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

curr_dir = './experiment_data/vowel_2/trained_circuits'
num_qubits = 4
meas_qubits = [0]

x_train, y_train, x_test, y_test = load_dataset('vowel_2', 'angle', 1)

# class_0_sel = np.random.choice(2000, 16, False)
# class_1_sel = np.random.choice(2000, 16, False) + 2000
# class_2_sel = np.random.choice(2000, 16, False) + 4000
# class_3_sel = np.random.choice(2000, 16, False) + 6000
# sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

class_0_sel = np.random.choice(300, 16, False)
class_1_sel = np.random.choice(300, 16, False) + 300
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]

num_params = 32
num_data = 32

d2_min_scores = []
d2_mean_scores = []
d2_var_scores = []
    
d2_t_min_scores = []
d2_t_mean_scores = []
d2_t_var_scores = []

mean_mat_scores = []
mean_t_mat_scores = []

# np.savetxt(curr_dir + '/fid_mats_sel_data.txt', sel_data)

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)
    
    batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                        weights_bounds, meas_qubits, 'matrix') 
    
    params = 2 * np.pi * np.random.sample((num_params, weights_bounds[-1]))
    
    if not os.path.exists(circ_dir + '/fid_mats'):
        os.mkdir(circ_dir + '/fid_mats')
    
    np.savetxt(circ_dir + '/fid_mats/metric_params.txt', params) 
    
    circ_d2_scores = []
    circ_d2_t_scores = []
    circ_mean_mat = np.zeros((num_data, num_data))
    circ_t_mean_mat = np.zeros((num_data, num_data))
    
    for j in range(num_params):
        curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, weights_bounds[-1]))
        mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
        
        t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
        diff_mat = mat - ideal
        diff_2d = np.sum(np.multiply(diff_mat, diff_mat))
            
        diff_t_mat = t_mat - ideal
        diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))
            
        circ_d2_scores.append(diff_2d)
        circ_d2_t_scores.append(diff_2d_t)
            
        circ_mean_mat += mat / num_params
        circ_t_mean_mat += t_mat / num_params

    np.savetxt(circ_dir + '/fid_mats/d2_scores.txt', circ_d2_scores)
    np.savetxt(circ_dir + '/fid_mats/d2_t_scores.txt', circ_d2_t_scores)
        
    d2_min_scores.append(np.min(circ_d2_scores))
    d2_mean_scores.append(np.mean(circ_d2_scores))
    d2_var_scores.append(np.var(circ_d2_scores))
        
    d2_t_min_scores.append(np.min(circ_d2_t_scores))
    d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
    d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    diff_mean_mat = ideal - circ_mean_mat
    diff_t_mean_mat = ideal - circ_t_mean_mat
        
    mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
    mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))     
    
    print(i)
        
np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27